---
Modeling the relationship between urban tree canopy, land cover, and land surface temperature
---

# Introduction
This exercise first shows some exploratory tasks to model land surface temperature (LST) and tree canopy to explore Urban Heat Island. As [described here by NASA](https://earthobservatory.nasa.gov/global-maps/MOD_LSTD_M), "land surface temperature is how hot the 'surface' of the Earth would feel to the touch in a particular location". It also lays out an initial modeling approach that combines exploratory data analysis with [KNN regresson](https://towardsdatascience.com/the-basics-knn-for-classification-and-regression-c1e8a6c955) and [decision tree models](https://towardsdatascience.com/decision-trees-in-machine-learning-641b9c4e8052) to better understand the patterns of the data and for prediction purposes in the R environment.  

We will use three primary datasets. The first are [land surface temperature (LST) data](https://www.usgs.gov/centers/eros/science/usgs-eros-archive-landsat-archives-landsat-level-2-provisional-surface?qt-science_center_objects=0#qt-science_center_objects) taken from the USGS EarthExplorer repository’s Analysis Ready Data series. These were processed to derive mean, minimum, and maximum LST in degrees Fahrenheit for each Census block group based on the 30 x 30 meter grid cells from the source Landsat images that fall within the boundary of the block group. 

We also calculated the percentage of each block group comprised by the [land cover designations](https://www.mrlc.gov/data/legends/national-land-cover-database-2016-nlcd2016-legend) in the 2016 National Land Cover Database maintained by the U.S. Geological Survey (USGS). 

Finally, we collected tree canopy data for Washington, DC from [Open Data DC](https://opendata.dc.gov/datasets/DCGIS::urban-tree-canopy-2020/about). Some landscape metrics are already calculated from the tree canopy data and will be used in the modeling exercise here. You can find the details on landscape metrics [here](https://cran.r-project.org/web/packages/landscapemetrics/landscapemetrics.pdf). This exercise focuses on the machine learning modeling and not on data collection from satellite imagery or tree canopy data. Only curated data files are shared here without going to the details of data curation process.

Before we dive into the modeling component, let's explore the data a little. 

The first code chunk is where we will load all the packages that we will need. **Please note** that you may need to install some of the packages below before you can load them with the `library` function. If you find any of the packages already not installed, uncomment (i.e., remove #) the install.packages command for that package. 

In [ ]:
#install.packages("caret", repos="http://cran.r-project.org")
#install.packages("ggmap", repos="http://cran.r-project.org")
#install.packages("rpart", repos="http://cran.r-project.org")
#install.packages("rpart.plot", repos="http://cran.r-project.org")
#install.packages("rsample", repos="http://cran.r-project.org")
#install.packages("tidyverse", repos="http://cran.r-project.org")
#install.packages("DataExplorer")
#install.packages("corrplot")
#install.packages("randomForest")

library(corrplot)
library(ggplot2)
library(caret)
library(ggmap)
library(rpart)
library(rpart.plot)
library(rsample)
library(sf)
library(tidyverse)
library(DataExplorer)
library(corrplot)
library(ModelMetrics)
library(randomForest)

Now we will read the required data files. WashingtonDC.csv file contains data on land surface temperature, land cover, and different landscape metrics for tree canopy at the census block group level. The Cenesus Block Group map is saved as an **sf** object called `dc.bgs` which we will use for visualizing heat exposure, land cover, and tree canopy coverage in the city.

In [ ]:
dc.data <- read.csv("data/WashingtonDC.csv")
dc.bgs <- st_read("data/WashingtondcBG2.shp")
head(dc.data)

Let's do some quick exploratory data analysis before proceeding. The following code chunk will create map for mean land surface temperature at census block group level.

In [ ]:
# Read the DC basemap so we do not have to retrieve it from 
# the web (i.e., use a Google API key each time)

load(file = "data/dc_basemap.RData")

dc.bgs.wgs84 <- st_transform(dc.bgs, 4326)
dc.bgs.wgs84 <- dc.bgs.wgs84 %>% mutate(GEOID.dbl = as.numeric(GEOID10))

dc.bg.data.wgs84 <- full_join(dc.bgs.wgs84, dc.data, by = c("GEOID.dbl" = "GEOID"))
dc.bg.data.wgs84 <- st_transform(dc.bg.data.wgs84, 4326)

# Recode NA to zero for NLCD variables
dc.bg.data.wgs84 <- dc.bg.data.wgs84 %>% replace(is.na(.), 0)

# Add categorical populations variable based on terciles
dc.bg.data.wgs84 <- dc.bg.data.wgs84 %>% 
    filter(!is.na(TotPop)) %>%
    mutate(Pop_Tercile = case_when(
      TotPop < 1125 ~ "Low",
      TotPop < 1700 & TotPop >= 1125 ~ "Middle",
      TotPop >= 1700 ~ "High"))

mean.lst.map <- ggmap(dc_basemap) +
  geom_sf(data = dc.bg.data.wgs84, aes(fill = MeanF), alpha = 0.4, inherit.aes = FALSE) +
    scale_fill_gradient(low = "green", high = "red") +
    theme_classic() +
    theme(axis.line = element_blank(), axis.text = element_blank(),
        axis.ticks = element_blank(), axis.title = element_blank()) + 
    labs(fill = "Mean LST °F")

mean.lst.map

Now we will create maps for high intensity and medium intensity developments to visually compare them with temperature map.

In [ ]:
dev.high.map <- ggmap(dc_basemap) +
  geom_sf(data = dc.bg.data.wgs84, aes(fill = Dev_HighIntensity), alpha = 0.5, inherit.aes = FALSE) +
    scale_fill_gradient(low = "green", high = "purple") +
    theme_classic() +
    theme(axis.line = element_blank(), axis.text = element_blank(),
        axis.ticks = element_blank(), axis.title = element_blank()) + 
    labs(fill = "% High Intensity Developed")

dev.med.map <- ggmap(dc_basemap) +
  geom_sf(data = dc.bg.data.wgs84, aes(fill = Dev_MedIntensity), alpha = 0.5, inherit.aes = FALSE) +
    scale_fill_gradient(low = "green", high = "purple") +
    theme_classic() +
    theme(axis.line = element_blank(), axis.text = element_blank(),
        axis.ticks = element_blank(), axis.title = element_blank()) + 
    labs(fill = "% Medium Intensity Developed")

dev.high.map
dev.med.map

Check out the variables available in the provided data. You will see that we have data on LST (MeanF, StdDevF, MinF, MaxF), along with tree canopy landscape metrics (ai, area_mn, etc.) and land cover classes (Dev_OpenSpace, Dev_LowIntensity, etc.) at the Block Group level.

In [ ]:
colnames(dc.bg.data.wgs84)

Now we will create histograms of the LST variables and a correlation plot (or correlogram) for all continuous LST variables with average patch size "area_mn" of tree canopy. It shows a significant negative correlation between mean temperature (MeanF) and average patch size of tree canopies.

In [ ]:
plot_histogram(dc.bg.data.wgs84[ , 39:42])
res <- cor(st_drop_geometry(dc.bg.data.wgs84[ , c(39:42, 19)]), use = "pairwise.complete.obs")
round(res, 2)
corrplot(res, type = "upper", order = "hclust", 
         tl.col = "black", tl.srt = 45)

Following code chunks create plots showing the distribution of land surface temperature with high, medium, and low intensity development, and how mean patch size of tree canopy are distributed at census block group level. You will see how the size of tree canopy varies by development intensity at the block group level.

In [ ]:
high.dev.lst.plot <- dc.bg.data.wgs84 %>%
  filter(!is.na(Dev_HighIntensity)) %>%
  ggplot(aes(x = MeanF, y = Dev_HighIntensity)) + 
  geom_point(aes(size = area_mn), color="red", alpha = 0.4) +
  geom_smooth(method=lm, se=FALSE, color = "purple", linetype="dashed") +
  theme_minimal() +
  labs(title = "", x = "Mean LST °F", y = "% High Intensity Developed", size = "Mean Patch 
       Size") 

high.dev.lst.plot

In [ ]:
med.dev.lst.plot <- dc.bg.data.wgs84 %>%
  filter(!is.na(Dev_MedIntensity)) %>%
  ggplot(aes(x = MeanF, y = Dev_MedIntensity)) + 
  geom_point(aes(size = area_mn), color = "#009E73", alpha = 0.4) +
  geom_smooth(method=lm, se=FALSE, color = "purple", linetype="dashed") +
  theme_minimal() +
  labs(title = "", x = "Mean LST °F", y = "% Medium Intensity Developed", size = "Mean Patch 
       Size") 

med.dev.lst.plot

In [ ]:
low.dev.lst.plot <- dc.bg.data.wgs84 %>%
  filter(!is.na(Dev_LowIntensity)) %>%
  ggplot(aes(x = MeanF, y = Dev_LowIntensity)) + 
  geom_point(aes(size = area_mn), color = "steelblue", alpha = 0.4) +
  geom_smooth(method=lm, se=FALSE, color = "purple", linetype="dashed") +
  theme_minimal() +
  labs(title = "", x = "Mean LST °F", y = "% Low Intensity Developed", size = "Mean Patch 
       Size") 

low.dev.lst.plot

Let's look at the summary statistics and histogram for all continuous landscape variables in the dataset.

In [ ]:
summary((dc.bg.data.wgs84[18:29]))
plot_histogram(dc.bg.data.wgs84[ , 18:29])

We can create a correlation matrix and correlogram of all the landscape metrics and mean LST to see which tree canopy landscape variables are significantly correlated with LST.

In [ ]:
res <- cor(st_drop_geometry(dc.bg.data.wgs84[ , c(17:21, 23:33, 39)]), use = "pairwise.complete.obs")
round(res, 2)
corrplot(res, type = "upper", order = "hclust", 
         tl.col = "black", tl.srt = 45)

The landscape metric correlation matrix plot shows most of the landscape metrics are negatively correlated with mean LST, but it also shows a positive relationship between mean LST and: (1) the landscape division index "division", (2) patch density "pd", and (3) the splitting index "split".

[Patch density](https://r-spatialecology.github.io/landscapemetrics/reference/lsm_l_pd.html) measures the fragmentation of tree canopy in the block group, while [division](https://r-spatialecology.github.io/landscapemetrics/reference/lsm_l_division.html) captures evenness of distribution across block groups. The [splitting index](https://r-spatialecology.github.io/landscapemetrics/reference/lsm_l_split.html) reflects variation in the size of patches. 

So, as fragmentation of tree canopy **(r = 0.72)**, unequal distribution of tree canopy **(r = 0.72)**, and the variation in the size of tree canopy patches **(r = 0.48)** increase, mean land surface temperature also increases. 

Mean LST has the highest negative relationship with: (1) average patch size "area_mn", (2) the aggregation index "ai", (3) patch cohesion index "cohesion", and (4) largest patch index "lpi". 

[Average patch size](https://r-spatialecology.github.io/landscapemetrics/reference/lsm_l_area_mn.html) is self-explanatory, the [aggregation index](https://link.springer.com/article/10.1023/A:1008102521322) and the [cohesion index](https://r-spatialecology.github.io/landscapemetrics/reference/lsm_l_cohesion.html) are measures of connectivity and compactness, and the [largest patch index](https://r-spatialecology.github.io/landscapemetrics/reference/lsm_l_lpi.html) is the percentage of the block group covered by the largest patch of tree canopy. 

*As the size and connectivity of tree canopy patches increases, mean land surface temperature decreases.* We can see this effect (i.e., mean tree canopy patch size) in the scatterplots generated by the code block above. The relationship between block group population and mean LST is weak **(r = 0.08)**.

Now that we have a feel for the distribution of LST and developed land uses from the NLCD data shown in the maps generated by the code block above, we can turn our attention to fitting models. We will begin with a quick look at the K-Nearest Neighbors technique for predicting the mean land surface temperature at the Census block group level in Washington DC.

## Fitting K-Nearest Neighbors Regression Models

As described in more detail here the [K-Nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) is one of the simplest prediction techniques out there. If we are trying to predict a categorical (qualitative) outcome or dependent variable, we are fitting a **classification** model. In these instances, the algorithm: 

  + Identifies K observations that have similar characteristics (e.g., independent variables)
  + Determines the majority (most common) class for those K similar observations and assigns that class as the predicted outcome
<br> 

If we are trying to predict a continuous (quantitative) outcome or dependent variable, we are fitting a **regression** model. In these instances, the algorithm: 

  + Identifies K observations that have similar characteristics (e.g., independent variables)
  + Determines the mean of the dependent variable values for those K similar observations and assigns that value as the predicted outcome
<br> 


In the code chunk below, we use the `set.seed` function to assign a starting value to the random number generator built into R so that we can replicate the analysis later.  

In [ ]:
set.seed(227)

dc.bg.data.no.geom <- st_drop_geometry(dc.bg.data.wgs84)

Now we will partition the data to training and testing sets using initial_split() function of rsample package.

In [ ]:
dc.bg.data.wgs84.split <- initial_split(dc.bg.data.no.geom, prop = 0.7)
dc.bg.data.wgs84.train <- training(dc.bg.data.wgs84.split)
dc.bg.data.wgs84.test  <- testing(dc.bg.data.wgs84.split)

We will extract the labels so that we can reattach predicted values to geometry later.

In [ ]:
train.labels <- as.numeric(unlist(labels(dc.bg.data.wgs84.train)[1]))
test.labels <- as.numeric(unlist(labels(dc.bg.data.wgs84.test)[1]))

dc.train.bgs <- dc.bg.data.wgs84[train.labels, ]
dc.test.bgs <- dc.bg.data.wgs84[test.labels, ]

Next, we will choose the variables we want to use for the model.

In [ ]:

dc.bg.data.wgs84.test <- subset(dc.bg.data.wgs84.test, select = c("MeanF", "TotPop", "ai", "area_mn", "cohesion", 
                                                                  "lpi", "division", "pd", "split", 
                                                                  "Water", "Dev_OpenSpace", "Grassland"))

dc.bg.data.wgs84.train <- subset(dc.bg.data.wgs84.train, select = c("MeanF", "TotPop", "ai", "area_mn", "cohesion", 
                                                                    "lpi", "division", "pd", "split", 
                                                                    "Water", "Dev_OpenSpace", "Grassland"))

For regression (i.e., continuous dependent variables), the KNN algorithm finds the K observations most like a given new observation in terms of its characteristics (independent variables) and assigns the mean of the dependent variable values of those neighbors to the new observation for which we are making a prediction.

In the code chunk below, we will use the **caret** package to fit a regression model using the KNN algorithm where the outcome is mean land surface temperature at the Census block group level. Performing K-Nearest Neighbors regression with the **caret** package has a few advantages including: (1) that it automatically tests various values of K for us, (2) it chooses the value of K that minimizes cross-validation error, and (3) fits the final, best-fitting model to the data. However, before running the code chunk below, take a look at the help documentation for the `caret::train` and the `caret::trainControl` functions. 

<br> 

The `trainControl` function can be accessed directly or from within the `train` function via the `trControl` argument. In the example below we are setting up [10-fold cross-validation](https://towardsdatascience.com/why-and-how-to-cross-validate-a-model-d6424b45261f) which happens to be one of the more commonly used. This means the algorithm will use 90 percent of the data to create a model and 10 percent to test it. Then, we will run the model again with a different 10 percent and repeat this 10 times, until all the data has been used as both training and test data. 

The `preProcess` argument can be used to scale, center, or otherwise transform the input data. The `tuneLength` parameter tells the algorithm to try different default values and the `tuneGrid` parameter lets us decide which values the main parameter will take: `tuneGrid = expand.grid(k = c(5, 11, 21, 25))`. This is how you could fit a different model than the "best-fitting" one identified by the algorithm. 

In [ ]:
# Set options for the cross-validation routine with 10-folds
ctrl <- trainControl(method="cv", number = 10)

# Fit the model...
knnFit.value <- train(MeanF ~ .,
                data = dc.bg.data.wgs84.train, method = "knn", trControl = ctrl,
                preProcess = c("center","scale"), tuneLength = 30)

# Examine the output
knnFit.value
plot(knnFit.value)

# Try again using a different resampling method where
# 10-folds are repeated 3 times
ctrl <- trainControl(method="repeatedcv", repeats = 3)
knnFit.value <- train(MeanF ~ ., 
                data = dc.bg.data.wgs84.train, method = "knn", trControl = ctrl, 
                preProcess = c("center","scale"), tuneLength = 30)
knnFit.value
plot(knnFit.value)

get_best_result = function(caret_fit) {
  best = which(rownames(caret_fit$results) == rownames(caret_fit$bestTune))
  best_result = caret_fit$results[best, ]
  rownames(best_result) = NULL
  best_result
}

get_best_result(knnFit.value)

We can more or less tell at which value of K the overall accuracy peaks (i.e., RMSE is lowest) from the graphic shown above. However, we can also retrieve this value directly and use it to predict mean LST for the test set we created before. The latter portions of the code below are [taken from this post](http://zevross.com/blog/2017/09/19/predictive-modeling-and-machine-learning-in-r-with-the-caret-package) and may be a useful reference. 

Print the best tuning parameter k that maximizes model accuracy

In [ ]:
knnFit.value$bestTune

Now use the best fitting model to predict mean LST for the test set. 

In [ ]:
knnPredict.value <- predict(knnFit.value, newdata = dc.bg.data.wgs84.test)
plot(knnPredict.value)

Now we can calculate RMSE between actual and predicted values.

In [ ]:
rmse(predicted = knnPredict.value, actual = dc.bg.data.wgs84.test$MeanF)

We can also compare OLS and random forest models. The following code chunk will compare the results from OLS, random forest, and KNN models. We will also check if the differences are statisticaly significant.

In [ ]:
lm.value <- train(MeanF ~., data = dc.bg.data.wgs84.train, method = "lm")
rf.value <- train(MeanF ~., data = dc.bg.data.wgs84.train, method = "rf")
knn.value <- train(MeanF ~ ., data = dc.bg.data.wgs84.train, method = "knn")

# Compare models
model_list <- list(lm = lm.value, rf = rf.value, knn = knn.value)
res <- resamples(model_list)
summary(res)

compare_models(lm.value, rf.value)
compare_models(rf.value, knn.value)

### Fitting Regression Tree Models 

The KNN approach is simplistic, but its results are often used as input (i.e., another independent variable) to more sophisticated models. Next, let's use the **rpart** package to fit **tree** models to our data. You can learn more about Regression Tree and other Decision Tree approaches for machine learning [here](https://www.datacamp.com/tutorial/decision-trees-R). Take a few moments to review the help documentation for the `rpart` and the `rpart.object` functions available [here](https://cran.r-project.org/web/packages/rpart/rpart.pdf), then proceed with the code chunk below. 

In [ ]:
# Because our dependent variable is continuous, we set 
# the methods argument to "anova" for regression...
model.lst <- rpart(
  formula = MeanF ~ .,
  data    = dc.bg.data.wgs84.train,
  method  = "anova", 
  minsplit = 4, 
  minbucket = 2
)

Let's examine the output in both text and visual format.

In [ ]:
model.lst
rpart.plot(model.lst, box.palette = "GnYlRd")

We can also retrieve more detailed info about the model.

In [ ]:
summary(model.lst)

We can also display the model's frame attribute.

In [ ]:
model.lst$frame

In this simple model, we begin with 315 observations in the training set and there are five internal nodes and six terminal nodes or leaves. As we move down the tree, the plot communicates which independent variables were chosen for splits, the values of those independent variables that optimized the split criterion (e.g., [**Gini impurity**](https://blog.quantinsti.com/gini-index/)), the average value of the independent variable in question at each node, and the percentage of the observations that "flow" to each node. 

As shown above, typing the name of the **rpart object** displays model information that mirrors what we see in the tree plot, but in text form. Note how the depth of the indent reflects the structure of the (plotted) tree. These are the elements shown above: 
    + node: a unique number for the node in the tree
    + split: the equation used to branch at the node
    + n: number of observations that "flow" through that branch
    + deviance (for regression): the deviance associated with that branch 
    + yval: predicted value at the node
    + *: an asterisk next to a node indicates it is terminal

Applying the `summary` function to an **rpart object** displays more detailed information. As shown in the documentation for the `rpart.object` function, the frame attribute of an rpart object (i.e., **model.lst$frame**) is a "data frame with one row for each node in the tree." The **var** column lists variables used at each split, **n** is the number of observations reaching that node, **wt** is the sum of of case weights (1 by default) for observations reaching that node, **dev** is the deviance which is a goodness of fit statistic that equals the sum of squared residuals for linear models. The **yval** column is the fitted value of the dependent variable at the node, **complexity** is the complexity parameter at which this split will collapse, **ncompete** is the number of competitor splits recorded, and **nsurrogate** is the number of surrogate splits recorded. 

Surrogate variables are used when a value is missing and so a split cannot be completed with the main variable. A surrogate is a different variable that is chosen to approximate the first-choice variable in a split. 

We can retrieve the **cp** value for the model and set it to a value above the threshold for the second **area_mn** split to demonstrate its effects like this: `model.lst$control$cp`

In [ ]:
# Prune the model...
model.pruned <- prune(model.lst, cp = 0.02)
model.pruned$frame
rpart.plot(model.pruned, box.palette = "GnYlRd")


# We could also set this parameter explicitly and rerun the model: 
temp <- rpart.control(cp = 0.02)
model.lst.2 <- rpart(
  formula = MeanF ~ .,
  data    = dc.bg.data.wgs84.train,
  method  = "anova",
  control = temp
)

model.lst.2$frame
rpart.plot(model.lst.2, box.palette = "GnYlRd")

According to the `rpart.object` documentation, the **cp table** is *"a matrix of information on the optimal prunings based on a complexity parameter"*. Behind the scenes `rpart` is automatically applying an array of cost complexity values to prune the tree. The **CP** column is the value of this parameter that would grow the tree to this size. The **nsplit** column is the number of nodes at that size, **rel error** is the error for predictions of the data that were used to estimate the model, while **xerror** is the cross-validation error generated by the `rpart` built-in cross validation routine and **xstd** gives us a sense of its variability over the default 10 iterations (see `plotcp`). Also note that (1 - relative error) is roughly equal to the variance explained by the model.

The **CP** values control the size of the tree and the greater the **CP** value, the fewer the number of splits in the tree. The optimal size of the tree is generally the row in the **CP table** that minimizes all error with the fewest branches. 

In the plot below, the dotted line refers to a recommendation from [Breiman et al. (1984)](https://www.taylorfrancis.com/books/mono/10.1201/9781315139470/classification-regression-trees-leo-breiman-jerome-friedman-olshen-charles-stone) to use the smallest tree within 1 standard deviation of the minimum cross validation error. You can think about the complexity parameter as the amount by which splitting that node improved the relative error. 

In [ ]:
model.lst$cptable
plotcp(model.lst)

model.lst$splits

The splits attribute consists of a row label which is the name of the split variable, and the rest of the columns are **count**, the number of observations sent left or right by the split (for competitor splits this is the number that would have been sent left or right had this split been used, for surrogate splits it is the number missing the primary split variable which were decided using this surrogate), **ncat** which is the number of categories or levels for the variable (+/-1 for a continuous variable), **improve** which is the improvement in deviance given by this split, or, for surrogates, the concordance of the surrogate with the primary, and **index**, the numeric split point. The last column **adj** gives the adjusted concordance for surrogate splits. For a continuous variable, the sign of **ncat** determines whether the subset x < cutpoint or x > cutpoint is sent to the left. 

A case weight is a nonnegative numeric variable that indicates the importance of each case. There are three types of case weights: frequencies, sampling weights, and variance weights.

Now we will use Regression Tree model for Prediction and check out the RMSE value.

In [ ]:
model.lst.pred <-  predict(model.lst, newdata = dc.bg.data.wgs84.test)

caret::RMSE(pred = model.lst.pred, obs = dc.bg.data.wgs84.test$MeanF)


Now we will create a couple of plots to compare the predicted values to the observed values (i.e., from the satellite imagery).

In [ ]:
dc.test.bgs$rtpred <- model.lst.pred

fitted <- ggplot() +
  geom_sf(data = dc.test.bgs, aes(fill = rtpred)) + theme_void() + 
  scale_fill_gradient(low = "yellow", high = "red") + 
  labs(title = "Predicted LST") + 
  theme(plot.title = element_text(hjust = 0.5))

fitted

observed <- ggplot() +
  geom_sf(data = dc.test.bgs, aes(fill = MeanF)) + theme_void() + 
  scale_fill_gradient(low = "yellow", high = "red") + 
  labs(title = "Satellite LST", fill = "Land Surface Temperature °F") +
  theme(plot.title = element_text(hjust = 0.5))

observed

We can also map the residuals to see where the model is performing poorly.

In [ ]:
dc.test.bgs$rtresid <- dc.test.bgs$MeanF - dc.test.bgs$rtpred

the.diff <- ggplot() +
  geom_sf(data = dc.test.bgs, aes(fill = rtresid)) + theme_void() + 
  scale_fill_gradient(low = "green", high = "red") + 
  labs(title = "Difference in Satellite and Predicted LST", fill = "°F") +
  theme(plot.title = element_text(hjust = 0.5),
        plot.margin = unit(c(1, 1, 2, 1), unit = "cm"))

the.diff

Based on the results from the above code chunk, this model does not fit the validation set **super** well. We could modify the predictors (independent variables) to try and improve our predictive capacity. 

We can also try other machine learning modeling approach for this analysis. The work of [Wilson et al. (2024)](https://journals.sagepub.com/doi/abs/10.1177/23998083241226848) is an example that modeled the relationship between urban tree canopy, landscape heterogeneity, and land surface temperature using Generalized Boosted Regression Model (GBM). 

<b> Reflection and challenge tasks </b>

This exercise shows several examples of machine learning approach to model the relationship between land surface temperature and tree canopy landscape metrices. It also shows how to compare the model results. This exercise can be further expanded by modeling data for another city (as done in [Wilson et al. (2024)](https://journals.sagepub.com/doi/abs/10.1177/23998083241226848)) or trying out different variable combinations for tree canopy metrics and land use classes as shared in the data files.